![title](vw.png)

https://github.com/VowpalWabbit/vowpal_wabbit

webspam: https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html

Реализация стохастического градиентного спуска для линейхных моделей, позволяющая запускаться на больших объёмах данных, за счет последовательной загрузки и обработки примеров.

Основной интерфейс работы -- shell

In [7]:
!python phraug/split.py webspam_wc_normalized_trigram.svm \
train_v.txt test_v.txt 0.8 dat_seed

P = 0.8
100000
200000
300000


### Формат входных данных

[Label] [Importance] [Tag]|Namespace Features |Namespace Features ... |Namespace Features

Namespace=String[:Value]

Features=(String[:Value] )*

где [] обозначает необязательные элементы, а (...)* означает повтор неопределенное число раз.

    Label - целевая переменная
    Importance - вес объекта
    Tag - является некоторой строкой без пробелов и отвечает за некоторое "название" примера, которое сохраняется при предсказании ответа
    Namespace - служит для группировки признаков. При использваонии в аргументах, Namespace именуются по первой букве
    Features - признаками объекта. Если признаком является строка, то она хешируется. Значение признака по умолчанию - 1, но его можно переопределить добавив значение после :. Все признаки не входящие в данную строку считаются равными 0.


Для приведения данных к необходимому виду, нужно либо создать отдельный файл, либо можем сразу писать в stdin

-b 29 – используем 29 бит для хэширования, то есть признаковое пространство ограничено признаками 2^29

--cache_file создаем на бинарный файл сразу из входных данных, что бы потом использовать при обучении

In [14]:
!python phraug/libsvm2vw.py train_v.txt /dev/stdout \
| vw -b 29 --cache_file train_v.cache

Num weight bits = 29
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = train_v.cache
Reading datafile = 
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0  -1.0000   0.0000     4173
0.500000 0.000000            2            2.0  -1.0000  -1.0000     2273
0.250000 0.000000            4            4.0  -1.0000  -1.0000     2241
0.866891 1.483782            8            8.0  -1.0000  -0.1026     3006
1.290152 1.713413           16           16.0   1.0000   1.0000     6257
1.184900 1.079648           32           32.0   1.0000   1.0000     2530
0.933104 0.681308           64           64.0   1.0000   1.0000     4209
0.657120 0.381136          128          128.0   1.0000   0.8641     3299
0.474147 0.291175          256          256.0   1.0000   1.0000     4314
0.365200 0.256252          512          512.0   1.0000  

In [15]:
!python phraug/libsvm2vw.py test_v.txt /dev/stdout \
| vw -b 29 --cache_file test_v.cache

Num weight bits = 29
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = test_v.cache
Reading datafile = 
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0  -1.0000   0.0000     1962
0.500002 0.000004            2            2.0  -1.0000  -0.9980     3854
1.073507 1.647012            4            4.0   1.0000  -0.8149     2560
0.809621 0.545736            8            8.0  -1.0000  -1.0000     4521
1.125993 1.442365           16           16.0   1.0000   1.0000     3004
0.926786 0.727578           32           32.0   1.0000   0.5664     4647
0.633341 0.339895           64           64.0   1.0000   1.0000     4331
0.572883 0.512425          128          128.0   1.0000   1.0000     4723
0.414914 0.256945          256          256.0   1.0000  -0.0028      943
0.358294 0.301673          512          512.0   1.0000   

### Запускаем обучение

Поддерживаемые функционалы (--loss_function):

* squared 1/2(y-a(x))^2
* classic - quadratic без взвешивания весов
* quantile \tau(a(x)-y)[y \leqslant a(x)]+(1-\tau)(y-a(x))[y \geqslant a(x)]
* logistic log(1 + exp (-ya(x)))
* hinge max(0,1 - ya (x))

Можно так же добваить регуляризацию:
* --l1 value
* --l2 value

In [16]:
!vw --cache_file train_v.cache --loss_function logistic -b 29 -P 10000 --passes 10

Num weight bits = 29
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = train_v.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.150932 0.150932        10000        10000.0   1.0000   9.3851     3702
0.126585 0.102238        20000        20000.0   1.0000   7.7036     4373
0.114448 0.090173        30000        30000.0  -1.0000  -5.2946     1346
0.108335 0.089996        40000        40000.0  -1.0000  -5.7050      117
0.103104 0.082181        50000        50000.0   1.0000  10.4773     2886
0.098748 0.076965        60000        60000.0  -1.0000  -2.9820      668
0.094184 0.066802        70000        70000.0   1.0000   7.5152     2739
0.089475 0.056509        80000        80000.0   1.0000  12.6887     3592
0.086110 0.059192        90000        90000.0  -1.0000  -7.6076     2184
0.081995 0.

0.041898 0.013588      1070000      1070000.0  -1.0000 -15.0180      214 h
0.041853 0.037023      1080000      1080000.0   1.0000   7.9168     1057 h
0.041643 0.019009      1090000      1090000.0   1.0000  11.0452     7637 h
0.041381 0.012816      1100000      1100000.0   1.0000  10.3532     5813 h
0.041293 0.031553      1110000      1110000.0   1.0000  15.0039     4431 h
0.041673 0.083899      1120000      1120000.0  -1.0000  -8.4450      259 h
0.042145 0.094979      1130000      1130000.0   1.0000   3.5971     2707 h
0.041989 0.024383      1140000      1140000.0   1.0000  18.7273     3596 h
0.042370 0.085791      1150000      1150000.0   1.0000  11.5160     5792 h
0.042187 0.021112      1160000      1160000.0   1.0000  14.6803     2786 h
0.042420 0.069491      1170000      1170000.0   1.0000   8.2518     1886 h
0.042415 0.041820      1180000      1180000.0  -1.0000 -14.9276     4418 h
0.042213 0.018383      1190000      1190000.0  -1.0000  -7.2060     1827 h
0.042074 0.025500      12

Параметр отвечающий, за наличие валидационного датасета 
* --holdout_off

In [17]:
!vw --cache_file train_v.cache --loss_function logistic -b 29 -P 10000 --passes 5 --holdout_off -f model_v_logistic

final_regressor = model_v_logistic
Num weight bits = 29
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = train_v.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.151009 0.151009        10000        10000.0   1.0000  12.1563     4848
0.128242 0.105475        20000        20000.0   1.0000  10.6945     5341
0.113506 0.084035        30000        30000.0   1.0000  12.5036     4536
0.103489 0.073437        40000        40000.0   1.0000   4.2872     6491
0.100947 0.090780        50000        50000.0  -1.0000 -10.1950      258
0.095359 0.067417        60000        60000.0  -1.0000  -2.4623      690
0.092400 0.074649        70000        70000.0  -1.0000  -6.4080     2107
0.088415 0.060522        80000        80000.0   1.0000   7.6411     3660
0.083498 0.044159        90000        90000.0  -1

0.021365 0.008450      1090000      1090000.0   1.0000  17.7296     6107
0.021229 0.006378      1100000      1100000.0   1.0000  12.0514     5898
0.021094 0.006255      1110000      1110000.0   1.0000  17.3003     4500
0.020959 0.006029      1120000      1120000.0   1.0000  20.6770     4504
0.020836 0.007019      1130000      1130000.0   1.0000  19.8030     4942
0.020705 0.005894      1140000      1140000.0   1.0000  18.8370     5609
0.020579 0.006276      1150000      1150000.0  -1.0000 -10.5862      795
0.020448 0.005372      1160000      1160000.0  -1.0000  -5.9244     4493
0.020319 0.005376      1170000      1170000.0  -1.0000  -8.7229      916
0.020190 0.005051      1180000      1180000.0  -1.0000 -10.8376      615
0.020064 0.005180      1190000      1190000.0   1.0000  17.8641     1086
0.019951 0.006467      1200000      1200000.0   1.0000  16.8653     5387
0.019832 0.005592      1210000      1210000.0   1.0000  28.5091     5572
0.019704 0.004267      1220000      1220000.0  -1.0

In [19]:
!vw --cache_file test_v.cache -i model_v_logistic -t -p preds.txt

only testing
predictions = preds.txt
Num weight bits = 29
learning rate = 0.5
initial_t = 0
power_t = 0.5
using cache_file = test_v.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
5.443122 5.443122            1            1.0  -1.0000  -3.3331     1962
31.637787 57.832451            2            2.0  -1.0000  -8.6048     3854
90.605680 149.573574            4            4.0   1.0000  11.6048     2560
177.015556 263.425431            8            8.0  -1.0000 -30.1791     4521
190.556256 204.096956           16           16.0   1.0000  19.7413     3004
222.444949 254.333641           32           32.0   1.0000  15.1369     4647
221.359732 220.274515           64           64.0   1.0000  19.2997     4331
197.487441 173.615151          128          128.0   1.0000  17.3244     4723
204.039426 210.591412          256          256

Вычислим метрики:

In [21]:
import sys
import numpy as np
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import confusion_matrix

y_file = "test_v.txt"
p_file = "preds.txt"

print "loading p..."

p = np.loadtxt( p_file )

y_predicted = np.ones(( p.shape[0] ))
y_predicted[p < 0] = -1

print "loading y..."

y = np.loadtxt( y_file, usecols= [0] )

print "accuracy:", accuracy( y, y_predicted )
print "AUC:", AUC( y, p )

print
print "confusion matrix:"
print confusion_matrix( y, y_predicted )

loading p...
loading y...
accuracy: 0.9946758265349935
AUC: 0.9984184361202244

confusion matrix:
[[27444   136]
 [  236 42054]]


Дополнительные параметры:
* -q AB -- Добавляет все парные признаки, где первый признак берется из Namespace A, а второй из Namespace B
* --cubic ABC -- Аналогично с тройками признаков
* --ngram AN -- генерирует n-граммы для пространста A
* --skips AK -- позволяет делать пропуски длины k в n-граммах простанств A

Автоматический подбор гиперпараметров можно делать через
**vw-hyperopt**

### Анализ модели

* --readable_model позволяет получить модель, в человеко-читаемом формате
* --invert_hash не умеет работать с кешем, но выдает поятные имена фичей, вместо номеров
* --audit_regressor позволяет получить описание модели с использованными фичами, однако требует повторного запуска. 

In [38]:
! vw -d test.vw -t -i small_model.vw --audit_regressor audit_regr

only testing
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
Regressor contains 8685 values
example      values         total   
counter      audited        progress
1                  300            3%
2                  324            3%
4                  352            4%
8                  369            4%
16                 471            5%
32                 690            7%
64                1060           12%
128               1246           14%
256               2265           26%
512               2780           32%
1024              4056           46%
2048              4774           54%
4096              5919           68%
7032              8685          100%


In [59]:
features = {}
with open ("audit_regr", "r") as audit:
    for l in audit:
        features[l.split (':')[0][5:]] = float(l.split (':')[2].strip ())

In [60]:
list (reversed (sorted(features, key=features.get)))

['ant',
 'rss',
 'email',
 'shy',
 'vip',
 'xml',
 '3jp',
 'narod',
 '0px',
 'steam',
 'notes',
 'imei',
 'netfirms',
 '1584',
 'eajaq',
 'torrent',
 'guest',
 'message',
 '376',
 'hotel',
 'tokio',
 'tristar',
 'milaya',
 'telefonnummer',
 'h2o',
 'copyright',
 'bmw',
 'hmsite',
 'domrabota',
 'autocad',
 'mop',
 'mdash',
 'published',
 'spb',
 '115',
 '143',
 '624',
 '644',
 'mot',
 '969ae6',
 '1987',
 'pronoun',
 'qip',
 '443',
 'admin',
 'mail',
 'net',
 'elit',
 'ins',
 'prostitutki',
 '2498',
 'tyumile',
 'antikombatz',
 'sisnbro',
 '1945',
 '377',
 'xtreemhost',
 'froebie',
 'waited',
 'acontario',
 'gidsupatscat',
 '5000',
 'appendchild',
 'design',
 'zona',
 '611',
 '116',
 '2125',
 'names',
 'ftp',
 '520',
 'ecce',
 '619',
 '184',
 '164',
 'padding',
 'edu',
 'lurkmore',
 'tcgkfnyj',
 '480',
 'novruslit',
 'incredimail',
 'nnn',
 'backup',
 'hosting',
 '2242px',
 '322',
 'anh',
 'fmneniewe',
 'ngerverkaufen',
 'studentenrevolte',
 'besitzergreifung',
 'leben',
 'rinderroulade

### Дополнительные параметры

* --progressive_loss прежде, чем пересчитать коэффициенты для текущего семпла, вычисляет для него функцию потерь. При множественных прохождениях теряет смысл.
* --bs позволяет использовать boostrapping

### Сcылки

https://habr.com/ru/company/ods/blog/326418/ 

http://fastml.com/vowpal-wabbit-liblinear-sbm-and-streamsvm-compared/

про значение весов: https://arxiv.org/abs/1011.1576

как сделан бустреп: https://arxiv.org/abs/1312.5021